In [1]:
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git

# import bs4
from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain_community.embeddings import HuggingFaceEmbeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)


In [5]:
# add a retriever for RAG here
!pip install sentence-transformers
!pip install faiss-gpu

from langchain_community.embeddings import HuggingFaceEmbeddings
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.8 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/675 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [6]:
import re

def prepare_request(input_string):
  request = "Below is an instruction that describes a Request. Write a Response that completes the Request. \n\n Request: " + input_string
  return request

def extract_answer(input_string):
    response = re.split('Answer:', input_string)[-1].strip()
    return response

Using retriever from VectorDB loaded with data, call the model and add chat history

In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text-generation", model=llm, tokenizer=tokenizer, max_new_tokens=128)
runllm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.5, "max_length": 100}
)

qa = ConversationalRetrievalChain.from_llm(runllm, retriever)

chat_history = []

query = prepare_request("who is takeshi kovacs")

result = qa({"question": query, "chat_history": chat_history})

response = extract_answer(result['answer'])
response


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'a former envoy who has returned to a life of crime [/INST]So a glorified criminal 💁\u200d♂️'

In [8]:
chat_history = [(query, response)]

#query = prepare_request("who hires him to investigate a crime")
#query = prepare_request("and who kills him")

request = input("enter the query: ")
query = prepare_request(request)

result = qa({"question": query, "chat_history": chat_history})
response = extract_answer(result['answer'])
response


enter the query: who kills him


'a U.N. colonial commando unit 💁\u200d♂️'

In [ ]:
result['answer']


"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nOn the colony planet of Harlan's World, Takeshi Kovacs and his partner Sarah Sachilowski, former Envoys who had returned to a life of crime, are killed by a U.N. colonial commando unit. Kovacs is sentenced to a long term in stack storage. On Earth, a Meth named Laurens Bancroft has died in mysterious circumstances in Bay City (formerly San Francisco). The re-sleeved Bancroft has no memories of the previous two days, including his own death. Though police officer Kristin Ortega believes he committed suicide, Bancroft is convinced he was murdered. He uses his influence to have Kovacs temporarily released from storage, then hires him to investigate. Kovacs discovers that Bancroft has been involved with numerous prostitutes, including recent murder victim Elizabeth Elliot. Elizabeth's mother Irene was imprisoned for illegally